## Overview
1. Assess Data Quality
2. Missing Values
3. Transform Features
4. Reshape (structure, granularity)

Quality checking (and fixing) and transformation are often cyclical: the quality checks point us toward transformations we need to make, and when we check the transformed columns to confirm that our data are ready for analysis, we may discover they need further cleaning.

### different expectations for quality

Data from a scientific experiment or study are typically clean, well-documented, and have a simple structure. These data are organized to be broadly shared so that others can build on or reproduce the findings. They are typically ready for analysis after little to no wrangling.

Data from government surveys often come with very detailed codebooks and meta data describing how the data are collected and formatted, and these datasets are also typically ready for exploration and analysis right out of the “box”.

Administrative data can be clean, but without inside knowledge of the source we may need to extensively check their quality. Also, since we often use these data for a purpose other than why they were collected in the first place, we may need to transform features or combine data tables.

Informally collected data, such as data scraped from the Web, can be quite messy and tends to come with little documentation. For example, texts, tweets, blogs, Wikipedia tables, etc. usually require formatting and cleaning to transform them into information ready for analysis.



## 9.1 Quality Check
Quality checks can reveal issues in the data that need to be addressed before proceeding with analysis.

### Four Vantage Points

#### 1. scope
Do the data match your understanding of the population?

#### 2. measurement


Considering what might be a _**reasonable value**_ for a feature.
- based on common knowledge of ranges
- documentation to tells us the expected values for a feature

Ensure that the **_data type_** matches our expectations
- a price to be a number
- weight values recorded in pounds, not kilograms

#### 3. Related Features
At times two features have builtin conditions on their values that we can cross-check for internal consistency. 

A cross-tabulation of the feature1 and feature2 can confirm constraint is met:

            `display_df(pd.crosstab(df[field1], df[field2]), rows=12)`

#### 4. Analysis
Even when data pass the previous quality checks, problems can arise with its usefulness:
- all but a handful of values for a feature are identical
- too many missing values, especially if there is a discernible pattern in the missing values
- a feature has many bad/corrupted values, then we might question the accuracy of even those values that fall in the appropriate range.

### Fixing or Not
what to do next when uncovering problems in the previous check

What ever approach you take, you will want to study the possible impact of the changes that you make on your analysis. For example, try to determine whether the records with corrupted values are similar to each other, and different from the rest of the data.

##### 1. Leave it as is
- Have discovered a characteristic of your data that will inform you about how to do your analysis and otherwise does not need correcting.

- The problem is relatively minor and most likely will not impact your analysis

- Replace corrupted values with `NaN`

##### 2. Modify Indiviudal Values
create a new feature with the modified value and preserve the original feature

##### 3. Remove a Column
If many values in a feature have problems, then consider eliminating that feature entirely. Rather than exclude a feature, there may be a transformation that allows you to keep the feature while reducing the level of detail recorded.

##### 4. Drop Records
DON'T:
- do not drop a large number of observations from a dataset without good reason
- do not simply correspond dropping records with corrupted values

DO:
- try to scale back your investigation to a particular **subgroup** of the data that is clearly defined by some criteria
- Even if an unusual value is in fact correct, you still might decide to exclude the record from your analysis because it’s so different from the rest of your data and you do not want it to overly influence your analysis.

## 9.2 Missing Values

1. population and access frame are not in alignment-->cannot access everyone to study
2. refuse to respond-->missing values and records-->results bias-->
    - rate is not negligible
    - differ in critical ways
    - increase the sample size can not reduce bias
    - even with efforts in preventing nonresponse, some amount of nonresponse is unavoidable

#### Field-Level
a record is not entirely missing, but a particular field in a record is unavailable
- special coding to specify the missing value: -99......

##### 1. missing completely at random
records with the missing data are like a randomly chosen subset of records
- whether or not a record has a missing value does not depend on the unobserved feature, the values of other features, or the sampling design

##### 2. missing at random given covariates (covariates are other features in data)

#### Further Categories 
- whether the respondent refused to answer
- was unsure of the answer
- or the interviewer didn’t ask the question

Each of these types of missing values is recorded using a different value.

#### Impute Missing Values (Imputation)
substitute a reasonable value for a missing one to create a “clean” data frame

##### 1. Deductive
fill in a value through logical relationships with other features

##### 2. Mean
uses an average value from rows in the dataset that aren’t missing

Downside: the variability in the imputed feature will be smaller because the feature now has values that are identical to the mean. 
For example: confidence intervals will be smaller than they should be (WHY?)

##### 3. Hot-Deck
uses a chance process to select a value at random from rows that have values

Downside: the strength of a relationship between the features might weaken because we have added randomness

##### Notice

For mean and hot-deck imputation, we often impute values based on other records in the dataset that have similar values in other features. More sophisticated imputation techniques use nearest-neighbor methods to find similar subgroups of records and others use regression techniques to predict the missing value.

With all of these types of imputation, we should **create a new feature** that contains the altered data or a new feature to indicate whether or not the response in the original feature has been imputed so that we can track our changes

 be sure to check the impact of dropping or changing features and records. And, be transparent and thorough in reporting any modifications you make to the data.

## 9.3 Transformation
There are many reasons a feature might need a transformation: the value codings might not be useful for analysis; we may want to apply a mathematical function to a feature; or we might want to pull information out of a feature and create a new feature.

#### 1. Type Conversion
convert the data from one format to another to make the data more useful for analysis

#### 2. Mathematical Transformation
a feature by extraction, where the new feature contains partial information taken from another

#### 3. Extraction
We can create a new feature that is True if the violation contains the word “vermin” in its text description and False otherwise.

This conversion of information to logical values (or 0-1 values) is extremely useful in data science.

### Transform Timestamp
Timestamps come in many different ways, we need to parse them for easier analysis

`pd.to_datetime()` method can parse the date strings into objects(a dtype of `datetime64[ns]`, which means that the values were successfully converted into `pd.Timestamp` objects), where we can pass in the format of the dates as a [date format string](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

The `pd.to_datetime()` method tries to automatically infer the timestamp format if we don’t pass in the `format=` argument. In many cases pandas will parse the timestamps properly. However, sometimes the parsing doesn’t output the correct timestamps (including this case) so we must explicitly specify the format.

pandas has special methods and properties for Series objects that hold timestamps using the `.dt accessor`